In [55]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [56]:
df = pd.read_csv("database.csv", low_memory=False)
print("Dataset Shape:", df.shape)

df.head()

Dataset Shape: (38113, 81)


,Vehicle ID,Year,Make,Model,Class,Drive,Transmission,Transmission Descriptor,Engine Index,Engine Descriptor,Engine Cylinders,Engine Displacement,Turbocharger,Supercharger,Fuel Type,Fuel Type 1,Fuel Type 2,City MPG (FT1),Unrounded City MPG (FT1),City MPG (FT2),Unrounded City MPG (FT2),City Gasoline Consumption (CD),City Electricity Consumption,City Utility Factor,Highway MPG (FT1),Unrounded Highway MPG (FT1),Highway MPG (FT2),Unrounded Highway MPG (FT2),Highway Gasoline Consumption (CD),Highway Electricity Consumption,Highway Utility Factor,Unadjusted City MPG (FT1),Unadjusted Highway MPG (FT1),Unadjusted City MPG (FT2),Unadjusted Highway MPG (FT2),Combined MPG (FT1),Unrounded Combined MPG (FT1),Combined MPG (FT2),Unrounded Combined MPG (FT2),Combined Electricity Consumption,Combined Gasoline Consumption (CD),Combined Utility Factor,Annual Fuel Cost (FT1),Annual Fuel Cost (FT2),Gas Guzzler Tax,Save or Spend (5 Year),Annual Consumption in Barrels (FT1),Annual Consumption in Barrels (FT2),Tailpipe CO2 (FT1),Tailpipe CO2 in Grams/Mile (FT1),Tailpipe CO2 (FT2),Tailpipe CO2 in Grams/Mile (FT2),Fuel Economy Score,GHG Score,GHG Score (Alt Fuel),My MPG Data,2D Passenger Volume,2D Luggage Volume,4D Passenger Volume,4D Luggage Volume,Hatchback Passenger Volume,Hatchback Luggage Volume,Start Stop Technology,Alternative Fuel/Technology,Electric Motor,Manufacturer Code,Gasoline/Electricity Blended (CD),Vehicle Charger,Alternate Charger,Hours to Charge (120V),Hours to Charge (240V),Hours to Charge (AC 240V),Composite City MPG,Composite Highway MPG,Composite Combined MPG,Range (FT1),City Range (FT1),Highway Range (FT1),Range (FT2),City Range (FT2),Highway Range (FT2)
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9001,(FFS),6.0,2.5,NaN,NaN,Regular,Regular Gasoline,NaN,17,0.0,0,0.0,0.0,0.0,0.0,24,0.0,0,0.0,0.0,0.0,0.0,21.0,34.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,1750,0,NaN,-2000,16.480500,0.0,-1,444.350000,-1,0.0,-1,-1,-1,N,74,7,0,0,0,0,NaN,NaN,NaN,NaN,False,NaN,NaN,0,0.0,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9005,(FFS) CA model,6.0,2.5,NaN,NaN,Regular,Regular Gasoline,NaN,17,0.0,0,0.0,0.0,0.0,0.0,24,0.0,0,0.0,0.0,0.0,0.0,21.0,34.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,1750,0,NaN,-2000,16.480500,0.0,-1,444.350000,-1,0.0,-1,-1,-1,N,74,7,0,0,0,0,NaN,NaN,NaN,NaN,False,NaN,NaN,0,0.0,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9002,(FFS),4.0,2.0,NaN,NaN,Regular,Regular Gasoline,NaN,18,0.0,0,0.0,0.0,0.0,0.0,25,0.0,0,0.0,0.0,0.0,0.0,23.0,35.0,0.0,0.0,21,0.0,0,0.0,0.0,0.0,0.0,1650,0,NaN,-1500,15.695714,0.0,-1,423.190476,-1,0.0,-1,-1,-1,N,0,0,0,0,0,0,NaN,NaN,NaN,NaN,False,NaN,NaN,0,0.0,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9006,(FFS) CA model,4.0,2.0,NaN,NaN,Regular,Regular Gasoline,NaN,18,0.0,0,0.0,0.0,0.0,0.0,25,0.0,0,0.0,0.0,0.0,0.0,23.0,35.0,0.0,0.0,21,0.0,0,0.0,0.0,0.0,0.0,1650,0,NaN,-1500,15.695714,0.0,-1,423.190476,-1,0.0,-1,-1,-1,N,0,0,0,0,0,0,NaN,NaN,NaN,NaN,False,NaN,NaN,0,0.0,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,NaN,1830,(FFS),4.0,2.5,NaN,NaN,Regular,Regular Gasoline,NaN,18,0.0,0,0.0,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,22.0,24.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,2050,0,NaN,-3500,19.388824,0.0,-1,522.764706,-1,0.0,-1,-1,-1,N,0,0,0,0,0,0,NaN,NaN,NaN,NaN,False,NaN,NaN,0,0.0,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0


In [57]:
df.info()

missing_pct = (df.isnull().sum().sort_values(ascending=False) / len(df) * 100).round(2)
missing_pct.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 81 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Vehicle ID                           38113 non-null  int64  
 1   Year                                 38113 non-null  int64  
 2   Make                                 38113 non-null  object 
 3   Model                                38113 non-null  object 
 4   Class                                38113 non-null  object 
 5   Drive                                36924 non-null  object 
 6   Transmission                         38102 non-null  object 
 7   Transmission Descriptor              15045 non-null  object 
 8   Engine Index                         38113 non-null  int64  
 9   Engine Descriptor                    22693 non-null  object 
 10  Engine Cylinders                     37977 non-null  float64
 11  Engine Displacement         

Alternate Charger              99.88
Vehicle Charger                99.87
Electric Motor                 98.42
Supercharger                   98.18
Range (FT2)                    96.24
Fuel Type 2                    96.23
Gas Guzzler Tax                93.98
Alternative Fuel/Technology    92.01
Turbocharger                   86.25
Start Stop Technology          83.10
Manufacturer Code              80.80
Transmission Descriptor        60.53
Engine Descriptor              40.46
Drive                           3.12
Engine Cylinders                0.36
Engine Displacement             0.35
Transmission                    0.03
GHG Score (Alt Fuel)            0.00
2D Luggage Volume               0.00
2D Passenger Volume             0.00
dtype: float64

In [58]:
#inspect unique fuel types
df["Fuel Type"].unique()

array(['Regular', 'Diesel', 'Premium', 'CNG', 'Electricity',
       'Gasoline or natural gas', 'Gasoline or E85',
       'Gasoline or propane', 'Premium or E85',
       'Premium Gas or Electricity', 'Midgrade',
       'Regular Gas and Electricity', 'Premium and Electricity',
       'Regular Gas or Electricity'], dtype=object)

In [59]:
#simplify fuel type
fuel_gas = {
    "Regular", "Premium", "Midgrade", "Diesel", "CNG",
    "Gasoline or natural gas", "Gasoline or E85", "Gasoline or propane",
    "Premium or E85"
}
fuel_hybrid = {
    "Premium Gas or Electricity", "Regular Gas and Electricity",
    "Premium and Electricity", "Regular Gas or Electricity"
}

df = df.copy()

df.loc[df["Fuel Type"].isin(fuel_gas), "Fuel Type"] = "Gas"
df.loc[df["Fuel Type"].isin(fuel_hybrid), "Fuel Type"] = "Hybrid"
df.loc[df["Fuel Type"].eq("Electricity"), "Fuel Type"] = "Electric"

df["Fuel Type"].unique()

array(['Gas', 'Electric', 'Hybrid'], dtype=object)

In [60]:
#simplify drive
drive_4 = {
    "4-Wheel or All-Wheel Drive", "4-Wheel Drive",
    "All-Wheel Drive", "Part-time 4-Wheel Drive"
}
drive_2 = {"2-Wheel Drive", "Front-Wheel Drive", "Rear-Wheel Drive"}

df.loc[df["Drive"].isin(drive_4), "Drive"] = "4-Wheel or All-Wheel Drive"
df.loc[df["Drive"].isin(drive_2), "Drive"] = "2-Wheel Drive"

# fill missing 'Drive' with the mode (most frequent)
if df["Drive"].isna().any():
    df["Drive"] = df["Drive"].fillna(df["Drive"].mode()[0])

df["Drive"].unique()

array(['2-Wheel Drive', '4-Wheel or All-Wheel Drive'], dtype=object)

In [61]:
#drop columns
cols_to_drop = [
    "Alternate Charger", "Vehicle Charger", "Electric Motor",
    "Supercharger", "Range (FT2)", "Fuel Type 2", "Gas Guzzler Tax",
    "Alternative Fuel/Technology", "Turbocharger", "Start Stop Technology",
    "Manufacturer Code", "Transmission Descriptor", "Engine Descriptor"
]

df = df.drop(columns=cols_to_drop, errors="ignore")
df.shape

(38113, 68)

In [62]:
#fill NaNs w/ most common value per column
for col in df.columns:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].mode()[0])

#verify nothing important missing
df.isnull().sum().head(20)

Vehicle ID                        0
Year                              0
Make                              0
Model                             0
Class                             0
Drive                             0
Transmission                      0
Engine Index                      0
Engine Cylinders                  0
Engine Displacement               0
Fuel Type                         0
Fuel Type 1                       0
City MPG (FT1)                    0
Unrounded City MPG (FT1)          0
City MPG (FT2)                    0
Unrounded City MPG (FT2)          0
City Gasoline Consumption (CD)    0
City Electricity Consumption      0
City Utility Factor               0
Highway MPG (FT1)                 0
dtype: int64

In [64]:
df.head()

,Year,Make,Model,Class,Drive,Transmission,Engine Cylinders,Engine Displacement,Fuel Type,City MPG (FT1),Highway MPG (FT1),Combined MPG (FT1),Tailpipe CO2 (FT1),Annual Fuel Cost (FT1),GHG Score
0,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,2-Wheel Drive,Manual 5-Speed,6.0,2.5,Gas,17,24,20,-1,1750,-1
1,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,2-Wheel Drive,Manual 5-Speed,6.0,2.5,Gas,17,24,20,-1,1750,-1
2,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,2-Wheel Drive,Manual 5-Speed,4.0,2.0,Gas,18,25,21,-1,1650,-1
3,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,2-Wheel Drive,Manual 5-Speed,4.0,2.0,Gas,18,25,21,-1,1650,-1
4,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,4.0,2.5,Gas,18,17,17,-1,2050,-1
